<a href="https://colab.research.google.com/github/kumudharam/Transformers/blob/main/sentence_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install timeline 
!pip install pipeline
!pip install pipelines 
!pip install openpyxl 
!pip install xlrd
!pip install xlsxwriter 

     |████████████████████████████████| 2.6 MB 5.6 MB/s 
     |████████████████████████████████| 895 kB 51.2 MB/s 
     |████████████████████████████████| 636 kB 57.9 MB/s 
     |████████████████████████████████| 3.3 MB 45.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for timeline: filename=timeline-0.0.7-py3-none-any.whl size=17005 sha256=101993f693783ad053d6ca683600730b16e32bd93ab23bf3b4469e6e57e95492
  Stored in directory: /root/.cache/pip/wheels/eb/03/7d/af6f2e055c7a95e6e9dcd5e64a9738678cac6bd8b05d4e1bd3
Successfully built timeline
     |████████████████████████████████| 161 kB 7.6 MB/s 
     |████████████████████████████████| 161 kB 53.1 MB/s 
     |████████████████████████████████| 161 kB 38.2 MB/s 
     |████████████████████████████████| 159 kB 49.9 MB/s 
     |████████████████████████████████| 159 kB 51.1 MB/s 
  Created wheel for pipelines: filename=pi

In [ ]:
!pip install pandas nltk spacy

In [ ]:
import nltk

# Download Punkt Sentence Tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os

# On Jupyter Notebook or Colab
DIR_PATH = os.getcwd()
# On Python module
#DIR_PATH = os.path.dirname(__file__)

FILE_PATH = os.path.join(DIR_PATH, "scripts.csv")

In [ ]:
import pandas as pd

# Read the file
df = pd.read_csv(r'/content/TestData.csv')
# Remove NaN values
df = df[~df["Dialogue"].isna()]
# Assign first_dialogue to the first row's "Dialogue" column
first_dialogue = df.loc[0, "Dialogue"]
print(first_dialogue)

GIVEN that I'm on any Screen of the application.


In [ ]:
df.head()

,Dialogue
0,GIVEN that I'm on any Screen of the application.
1,WHEN I fill the created before as like other f...
2,AND click the Apply filters to get the filtere...
3,THEN the application will display the data bas...
4,IF I close the application and Launch again.


In [ ]:
# use Python's split
first_dialogue.split(".")

["GIVEN that I'm on any Screen of the application", '']

In [ ]:
from nltk.tokenize import sent_tokenize

sent_tokenize(first_dialogue)

["GIVEN that I'm on any Screen of the application."]

In [ ]:
%%timeit -n 10
df.loc[:5000, "Dialogue"].apply(lambda x: sent_tokenize(x))

10 loops, best of 5: 584 µs per loop


In [ ]:
import spacy
# use spacy with the dependency parse 
nlp = spacy.load("en_core_web_sm")
[str(sent) for sent in nlp(first_dialogue).sents]

["GIVEN that I'm on any Screen of the application."]

In [ ]:
#%%timeit -n 10
# WARNING: takes a long time!
#df.loc[:5000, "Dialogue"].apply(lambda x: [sent.text for sent in nlp(x).sents])

In [ ]:
from spacy.lang.en import English
# use spacy with the sentencizer
nlp = English()  # just the language with no model
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)
[str(sent) for sent in nlp(first_dialogue).sents]

["GIVEN that I'm on any Screen of the application."]

In [ ]:
%%timeit -n 10
df.loc[:5000, "Dialogue"].apply(lambda x: [sent.text for sent in nlp(x).sents])

10 loops, best of 5: 758 µs per loop


In [ ]:
import re
# use regular expression
rule = r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"
re.split(rule, first_dialogue)

["GIVEN that I'm on any Screen of the application."]

In [ ]:
%%timeit -n 10
df.loc[:5000, "Dialogue"].apply(lambda x: re.split(rule, x))

10 loops, best of 5: 361 µs per loop


In [ ]:
%%timeit -n 10
# Without dependency parser
tokenized_data = []
first_5000_rows = df.loc[:5000, "Dialogue"]
for doc in nlp.pipe(first_5000_rows, batch_size=20):
  tokenized_data.append([sent.text for sent in doc.sents])

10 loops, best of 5: 470 µs per loop


In [ ]:
# Transform data using spaCy
nlp = spacy.load("en_core_web_sm")
# WARNING: takes a long time!
df["Dialogue"] = df["Dialogue"].apply(lambda x: [sent.text for sent in nlp(x).sents])

In [ ]:
df = df.explode("Dialogue", ignore_index=True)

In [ ]:
df.rename(columns={"Unnamed: 0": "Dialogue ID"}, inplace=True)
df.index.name = "Sentence ID"

In [ ]:
df.head()

,Dialogue
Sentence ID,
0,GIVEN that I'm on any Screen of the application.
1,WHEN I fill the created before as like other f...
2,AND click the Apply filters to get the filtere...
3,THEN the application will display the data bas...
4,IF I close the application and Launch again.


In [ ]:
Text = df.to_csv("scripts_tokenized.csv")
print(df.to_csv("scripts_tokenized.csv"))

None


In [ ]:
!pip install -U transformers==3.0.0 --quiet
!python -m nltk.downloader punkt

!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, sent_tokenize
from keras.preprocessing.text import Tokenizer

from transformers import pipeline
import timeline
import pandas as pd
import csv 
import openpyxl
import xlsxwriter
from itertools import chain 
workbook = xlsxwriter.Workbook('/content/TestData1.xlsx')
worksheet = workbook.add_worksheet()

# Read the file
df1 = pd.read_csv(r'/content/TestData.csv')
df = pd.DataFrame(df1, columns =['AccpCriteria'])
 # Iterate over the sequence of column names
for column in reversed(df.columns):
    
    # Select column contents by column
    # name using [] operator
    columnSeriesObj = df[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)

    Stringval = str(columnSeriesObj.values)
    print(Stringval)

from pipelines import pipeline
nlp = pipeline("question-generation") 

nlp(Stringval)


# Driver code
word = nlp(Stringval)
#word1 = split(word)
#returns only values

values = []
for dictionary in word:
     values.extend([v for k, v in dictionary.items() if k == 'answer'])
print(values)

def split(word):
    return [char for char in word]
word2 = split(values)
header1 = "Steps"
header2 = "Expected_result"
worksheet.write('A1', header1)
worksheet.write('B1', header2)

row = 1
column = 1
content = word2
  
# iterating through content list
for item in content :
    worksheet.write(row, column, item)
    row += 1

values1 = []
for dictionary in word:
     values1.extend([v for k, v in dictionary.items() if k == 'question'])
word2 = split(values1)

n = ["happens","do", "will", "is", "are", "if", "?", "[", "]", "``", "'", "'", "[ "]     
stopwords = set(n)
word_tokens = word_tokenize(str(values1))

new_strings = [w for w in word_tokens if not w.lower() in stopwords]

new_strings = []
for string in word_tokens:
  if string not in stopwords:
     
     #Prefix = "Check" 
     new_string = string.replace("What", "Check whether")
     new_strings.append(new_string)
     #listToStr = split(new_strings)
     listToStr = ' '.join(new_strings)
     
     listToStr1 = listToStr.split(', ')


print(listToStr1)

row = 1
column = 0
word3 = listToStr1

# iterating through content list
for item in word3:
    worksheet.write_string(row, column, item)
    row += 1
workbook.close()   




/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Cloning into 'question_generation'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 265 (delta 100), reused 95 (delta 95), pack-reused 156
Receiving objects: 100% (265/265), 289.06 KiB | 17.00 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_gene

In [15]:
 !pip install -U transformers==3.0.0 --quiet
!python -m nltk.downloader punkt

!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, sent_tokenize
from keras.preprocessing.text import Tokenizer

from transformers import pipeline
import timeline
import pandas as pd
import csv 
import openpyxl
import xlsxwriter
from itertools import chain 
workbook = xlsxwriter.Workbook('/content/TestCaseOutput.xlsx')
worksheet = workbook.add_worksheet()

# Read the file
df1 = pd.read_csv(r'/content/TestScenario8.csv')
df = pd.DataFrame(df1, columns =['AccpCriteria'])
 # Iterate over the sequence of column names
for column in reversed(df.columns):
    
    # Select column contents by column
    # name using [] operator
    columnSeriesObj = df[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)

    Stringval = str(columnSeriesObj.values)
    print(Stringval)

from pipelines import pipeline
nlp = pipeline("question-generation") 

nlp(Stringval)


# Driver code
word = nlp(Stringval)
#word1 = split(word)
#returns only values

values = []
for dictionary in word:
     values.extend([v for k, v in dictionary.items() if k == 'answer'])
print(values)

def split(word):
    return [char for char in word]
word2 = split(values)
header1 = "Steps"
header2 = "Expected_result"
worksheet.write('A1', header1)
worksheet.write('B1', header2)

row = 1
column = 1
content = word2
  
# iterating through content list
for item in content :
    worksheet.write(row, column, item)
    row += 1

values1 = []
for dictionary in word:
     values1.extend([v for k, v in dictionary.items() if k == 'question'])
word2 = split(values1)

n = ["happens","do", "will", "is", "are", "if", "?", "[", "]", "``", "'", "'", "[ "]     
stopwords = set(n)
word_tokens = word_tokenize(str(values1))

new_strings = [w for w in word_tokens if not w.lower() in stopwords]

new_strings = []
for string in word_tokens:
  if string not in stopwords:
     
     #Prefix = "Check" 
     #new_string = string.replace("What", "Check whether")
     new_strings.append(string)
     listToStr = ' '.join(new_strings)
     listToStr1 = listToStr.split(', ')
      
append_str = 'Check if '
pre_res = [append_str + sub for sub in listToStr1]
#suf_res = [sub + append_str for sub in values]
  
print("list after prefix addition : " + str(pre_res))

print(pre_res)

row = 1
column = 0
word3 = pre_res

# iterating through content list
for item in word3:
    worksheet.write_string(row, column, item)
    row += 1
workbook.close()   




/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Cloning into 'question_generation'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 265 (delta 100), reused 95 (delta 95), pack-reused 156
Receiving objects: 100% (265/265), 289.06 KiB | 4.74 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation
[nltk_data] Do